# Squin gates of interest

given immutable lists `q: IList[Qubit, Literal[7]]` of qubits, and `p: float` noise rate

### Depolarizing noise channels

In [ ]:
from bloqade import squin
from kirin.dialects.ilist import IList


# a squin kernel
@squin.kernel
def good_to_know(p: float = 0):
    q = squin.qalloc(3)  # allocate qubits

    # Single qubit noise with parameter p; broadcast applies channel in parallel to all qubits in list
    squin.broadcast.depolarize(p, IList([q[0], q[1]]))

    # Two-qubit noise
    squin.depolarize2(p, q[0], q[1])

### Measure and reset

In [ ]:
# measure
squin.broadcast.measure(q)

# reset
squin.broadcast.reset(q)

note that to implement resets and measurements on single qubits, just drop `.broadcast` and apply to single-qubit registers.

# From Bloqade Squin kernels to stabilizer samplers
### Convert circuit to Stim/Tsim, visualize, run sampler

In [ ]:
import bloqade.stim
import bloqade.tsim


@squin.kernel
def main():
    my_kernel()


# plotting circuit diagram (plotting is nicer on Tsim)
tsim_circ = bloqade.tsim.Circuit(main)
tsim_circ.diagram(height=400)

# sampling using Stim (Clifford-only sampling is faster on Stim)
stim_circ = bloqade.stim.Circuit(main)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=100)

# On noise 
### Using and adjusting heuristic noise model

In [ ]:
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit


# applying noise model to circuit according to different architectures; must use Cirq circuit
@squin.kernel
def main():
    my_kernel()


cirq_main = emit_circuit(main)  # emit to Cirq
noise_model = (
    noise.GeminiOneZoneNoiseModel()
)  # define noise model; here OneZone default
main_noisy = noise.transform_circuit(
    cirq_main, model=noise_model
)  # annotate circuit w noise
squin_main = load_circuit(main_noisy)  # back to Squin
noisy_stim = bloqade.stim.Circuit(squin_main)  # now to Stim


# Example 1: defining new noise model on top of OneZone architecture, scaling all noise parameters by a constant factor
noise_scale = 1
noise_model_1 = noise.GeminiOneZoneNoiseModel(scaling_factor=noise_scale)

# Example 2: changing just some of the channels: no local phase-z errors
noise_model_2 = noise.GeminiOneZoneNoiseModel(local_pz=0.0)